# Notebook 10: A hyperparameter search

### Goal: Show an example of hyperparameter tuning 

#### Background

If you ever choose to look at any of the ML method documentation, you will find there are alot of switches and nobs you can play with. See this page on ```RandomForestRegressor``` [click](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)

#### Step 1 & 2: Import packages and load data for Classification  
In Notebook 5 we only wanted 1 feature, no we want all available inputs (36 total). So all we need to change is the ```features_to_keep``` keyword to inlcude all indices. 